# Ollama Function Calling
- By Marcelo Rovai @26Sept24

## Importing Libraries

- **sys**: Provides access to system-specific parameters and functions. It's used to get command-line arguments.
- **haversine**: A function from the haversine library that calculates the distance between two geographic points using the Haversine formula.
- **openAI**: A module for interacting with the OpenAI API (although it's used in conjunction with a local setup, Ollama). Everything is off-line here.
- **pydantic**: Provides data validation and settings management using Python-type annotations. It's used to define the structure of expected response data.
- **instructor**: A module is used to patch the OpenAI client to work in a specific mode (likely related to structured data handling).

In [22]:
import sys
import time
from haversine import haversine
from openai import OpenAI
from pydantic import BaseModel, Field
import instructor

## Defining Input and Model

On a python string, it is possible get the country from command-line arguments

In this notebook, we should define the country name as a variable

In [2]:
country = "France"

In [34]:
MODEL = 'phi3.5:3.8b'   # The name of the model to be used
mylat = -33.33          # Latitude of Santiago de Chile
mylon = -70.51          # Longitude of Santiago de Chile

## Defining the Response Data Structure

**CityCoord**: A Pydantic model that defines the expected structure of the response from the LLM. It expects three fields: city (name of the city), lat (latitude), and lon (longitude).

In [4]:
class CityCoord(BaseModel):
    city: str = Field(..., description="Name of the city")
    lat: float = Field(..., description="Decimal Latitude of the city")
    lon: float = Field(..., description="Decimal Longitude of the city")

## Setting Up the OpenAI Client

In [5]:
client = instructor.patch(
    OpenAI(
        base_url="http://localhost:11434/v1",  # Local API base URL (Ollama)
        api_key="ollama",                      # API key (not used)
    ),
    mode=instructor.Mode.JSON,                 # Mode for structured JSON output
)

## Generating the Response

In [6]:
resp = client.chat.completions.create(
    model=MODEL,
    messages=[
        {
            "role": "user",
            "content": f"return the decimal latitude and decimal longitude of the capital of the {country}."
        }
    ],
    response_model=CityCoord,
    max_retries=10
)

In [7]:
resp

CityCoord(city='Paris', lat=48.864716, lon=2.335182)

In [8]:
resp.city, resp.lat, resp.lon

('Paris', 48.864716, 2.335182)

## Calculating the Distance

In [9]:
distance = haversine((mylat, mylon), (resp.lat, resp.lon), unit='km')
distance

11628.656266429276

In [10]:
print(f"Santiago de Chile is about {int(round(distance, -1)):,} kilometers away from {resp.city}.")

Santiago de Chile is about 11,630 kilometers away from Paris.


## Creating a function

In [27]:
def calc_dist(country, model=MODEL):
    
    start_time = time.perf_counter()  # Start timing

    class CityCoord(BaseModel):
        city: str = Field(..., description="Name of the city")
        lat: float = Field(..., description="Decimal Latitude of the city")
        lon: float = Field(..., description="Decimal Longitude of the city")

    client = instructor.patch(
        OpenAI(
            base_url="http://localhost:11434/v1",  # Local API base URL (Ollama)
            api_key="ollama",                      # API key (not used)
        ),
        mode=instructor.Mode.JSON,                 # Mode for structured JSON output
)
    
    resp = client.chat.completions.create(
    model=model,
    messages=[
        {
            "role": "user",
            "content": f"return the decimal latitude and decimal longitude of the capital of the {country}."
        }
    ],
    response_model=CityCoord,
    max_retries=10
    )

    distance = haversine((mylat, mylon), (resp.lat, resp.lon), unit='km')
    
    end_time = time.perf_counter()  # End timing
    elapsed_time = end_time - start_time  # Calculate elapsed time
    
    print(f"Santiago de Chile is about {int(round(distance, -1)):,} kilometers away from {resp.city}. [INFO] ==> {MODEL}): {elapsed_time:.1f} s")


In [35]:
calc_dist('france', model=MODEL)

Santiago de Chile is about 11,630 kilometers away from Paris. [INFO] ==> phi3.5:3.8b): 23.6 s


In [29]:
calc_dist('colombia')

Santiago de Chile is about 4,240 kilometers away from Bogotá. [INFO] ==> phi3.5:3.8b): 16.2 s


In [30]:
calc_dist('united states')

Santiago de Chile is about 8,060 kilometers away from Washington, D.C.. [INFO] ==> phi3.5:3.8b): 16.5 s


### Using Llama3.2:3B

In [31]:
MODEL = 'llama3.2:3B'
calc_dist('france')
calc_dist('colombia')
calc_dist('united states')

Santiago de Chile is about 11,630 kilometers away from Paris. [INFO] ==> llama3.2:3B): 20.9 s
Santiago de Chile is about 4,250 kilometers away from Bogota. [INFO] ==> llama3.2:3B): 21.6 s
Santiago de Chile is about 8,060 kilometers away from Washington. [INFO] ==> llama3.2:3B): 17.4 s


### Using Llama3.2:1B

In [32]:
MODEL = 'llama3.2:1B'
calc_dist('france')
calc_dist('colombia')
calc_dist('united states')

Santiago de Chile is about 11,630 kilometers away from Paris. [INFO] ==> llama3.2:1B): 16.4 s
Santiago de Chile is about 4,250 kilometers away from Bogota. [INFO] ==> llama3.2:1B): 17.0 s
Santiago de Chile is about 8,060 kilometers away from Washington D.C.. [INFO] ==> llama3.2:1B): 16.7 s


### Using gemma2:2b

In [33]:
MODEL = 'gemma2:2b'
calc_dist('france')
calc_dist('colombia')
calc_dist('united states')

Santiago de Chile is about 11,630 kilometers away from Paris. [INFO] ==> gemma2:2b): 22.0 s
Santiago de Chile is about 4,240 kilometers away from Bogota. [INFO] ==> gemma2:2b): 22.3 s
Santiago de Chile is about 8,060 kilometers away from Washington D.C.. [INFO] ==> gemma2:2b): 17.5 s
